In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
google_api_key = os.getenv("GOOGLE_API_KEY")

In [3]:
if google_api_key == "":
    print("api Key not found.")
else:
    print("api Key found.")
    

api Key found.


In [4]:
from llama_index.llms.gemini import Gemini
import google.generativeai as genai
from llama_index.core import SimpleDirectoryReader
from llama_index.core import VectorStoreIndex
from IPython.display import Markdown, display
from llama_index.core import ServiceContext
from llama_index.core import StorageContext, load_graph_from_storage
from llama_index.embeddings.gemini import GeminiEmbedding
from llama_index.core import Settings


c:\Users\DELL\qasystem\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
genai.configure(api_key=google_api_key)

In [6]:
for models in genai.list_models():
    print(models)

Model(name='models/embedding-gecko-001',
      base_model_id='',
      version='001',
      display_name='Embedding Gecko',
      description='Obtain a distributed representation of a text.',
      input_token_limit=1024,
      output_token_limit=1,
      supported_generation_methods=['embedText', 'countTextTokens'],
      temperature=None,
      max_temperature=None,
      top_p=None,
      top_k=None)
Model(name='models/gemini-1.0-pro-vision-latest',
      base_model_id='',
      version='001',
      display_name='Gemini 1.0 Pro Vision',
      description=('The original Gemini 1.0 Pro Vision model version which was optimized for '
                   'image understanding. Gemini 1.0 Pro Vision was deprecated on July 12, 2024. '
                   'Move to a newer Gemini version.'),
      input_token_limit=12288,
      output_token_limit=4096,
      supported_generation_methods=['generateContent', 'countTokens'],
      temperature=0.4,
      max_temperature=None,
      top_p=1.0,
     

In [7]:
for models in genai.list_models():
    if 'generateContent' in models.supported_generation_methods:
        print(models.name)

models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-flash-latest
models/gemini-1.5-flash
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-2.5-pro-exp-03-25
models/gemini-2.5-pro-preview-03-25
models/gemini-2.5-flash-preview-04-17
models/gemini-2.5-flash-preview-05-20
models/gemini-2.5-flash
models/gemini-2.5-flash-preview-04-17-thinking
models/gemini-2.5-flash-lite-preview-06-17
models/gemini-2.5-pro-preview-05-06
models/gemini-2.5-pro-preview-06-05
models/gemini-2.5-pro
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-preview-image-generation
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-flash-lite-preview
models/gemini-2.0-p

In [8]:
documents= SimpleDirectoryReader("../Data")

In [9]:
doc=documents.load_data()

In [10]:
print(doc[0].text)

BASICS OF MACHINE LEARNING

**Definition:**
Machine Learning (ML) is a subset of Artificial Intelligence (AI) that enables systems to learn from data and improve their performance without being explicitly programmed.

**Types of ML:**
1. Supervised Learning:
   - Learns from labeled data (input-output pairs)
   - Algorithms: Linear Regression, Logistic Regression, Decision Trees, SVM, k-NN, Random Forest, XGBoost
   - Applications: Spam detection, disease prediction, stock price forecasting

2. Unsupervised Learning:
   - Learns from unlabeled data
   - Algorithms: K-Means Clustering, Hierarchical Clustering, PCA, Autoencoders
   - Applications: Customer segmentation, anomaly detection

3. Semi-Supervised Learning:
   - Uses a small amount of labeled data with a large amount of unlabeled data
   - Used when labeling is expensive or time-consuming

4. Reinforcement Learning:
   - Learns through trial and error using rewards and punishments
   - Key Concepts: Agent, Environment, Actions,

In [11]:
model=Gemini(models='gemini-pro',api_key=google_api_key)

C:\Users\DELL\AppData\Local\Temp\ipykernel_11360\2941566521.py:1: DeprecationWarning: Call to deprecated class Gemini. (Should use `llama-index-llms-google-genai` instead, using Google's latest unified SDK. See: https://docs.llamaindex.ai/en/stable/examples/llm/google_genai/)
  model=Gemini(models='gemini-pro',api_key=google_api_key)


In [12]:
gemini_embed_model=GeminiEmbedding(model_name="models/embedding-001")

C:\Users\DELL\AppData\Local\Temp\ipykernel_11360\4148756438.py:1: DeprecationWarning: Call to deprecated class GeminiEmbedding. (Should use `llama-index-embeddings-google-genai` instead, using Google's latest unified SDK. See: https://docs.llamaindex.ai/en/stable/examples/embeddings/google_genai/)
  gemini_embed_model=GeminiEmbedding(model_name="models/embedding-001")


In [ ]:
# service_context = ServiceContext.from_defaults(llm=model, embed_model=gemini_embed_model, chunk_size=800, chunk_overlap=200)
# service context is deprecated, so we use Settings instead
Settings.llm = model
Settings.embed_model = gemini_embed_model
Settings.chunk_size = 800
Settings.chunk_overlap = 20

In [16]:
# index = VectorStoreIndex.from_documents(doc, service_context=service_context)
# VectorStoreIndex is a class 
# from_documents is a class method that creates an index from the documents
# we are reading our document here and creating an index from it

# Index creation
index = VectorStoreIndex.from_documents(doc, Settings=Settings)

In [ ]:
index
# it is an object that contains the index of the documents

In [18]:
index.storage_context.persist()
# it will create a local database in the current directory with the name storage 
# persist is a method that saves the index to the local storage

In [19]:
query_engine = index.as_query_engine()
# it is an object that allows us to query the index

In [ ]:
response=query_engine.query("What is Machine Learning?")
# this is a method that takes a query as input and returns a response

In [ ]:
print(response.response)
# this is the response that we get from the query

Machine Learning is a subset of Artificial Intelligence that enables systems to learn from data and improve their performance without being explicitly programmed.



In [23]:
response=query_engine.query("Who is Virat Kohli?")

In [24]:
print(response.response)

I am sorry, but the context does not contain information about Virat Kohli.

